# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmplot
import gmaps.datasets
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "c://Users/shawn/python_api_challenge//cities.csv"
city_data = pd.read_csv(file)
city_data.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

#Center on Algeria; zoom to level 2
fig = gmaps.figure(center = (28,1.6), zoom_level = 2)

#Create heatmap layer with data frame lat and long, weighted by Humidity
heatmap_layer = gmaps.heatmap_layer(
    city_data[['Lat', 'Lng']], weights=city_data['Humidity'],
    max_intensity=100, point_radius=15.0
)

#Add heatmap layer
fig.add_layer(heatmap_layer)

#Show map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Create new data frame
ideal_weather = city_data
#Filter data frame to show cities with max temp between 70 and 90 degrees
ideal_weather = city_data[city_data['Max Temp'].between(70,90)]
#Filter data frame to show cities with wind speed between 0 and 15 MPH
ideal_wind = ideal_weather[ideal_weather['Wind Speed'].between(0,15)]
#Filter data frame to show cities with cloudiness between 0% and 10%
ideal_cloudiness = ideal_wind[ideal_wind['Cloudiness'].between(0,10)]
#Drop NaN values
dream_weather = ideal_cloudiness.dropna()
#View data to confirm correct filtering
dream_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Add column placeholders for Hotel Name, Lat and Long values
hotel_df = pd.DataFrame(dream_weather)
hotel_df["Hotel Name"] = ""
hotel_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,


In [6]:
from api_keys import g_key

#Static parameters
params = {"types" : "lodging",
          "radius" : 5000,
          "key" : g_key}

#Use/iterate through lat/longs from data frame
for index, row in hotel_df.iterrows():

#set variables for Lat/Long values from data frame for iteration
    lat = row["Lat"]
    lng = row["Lng"]
    
#Change coordinates each iteration
    params["location"] = f"{lat},{lng}"
    
#Set url as API address
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#Make request from url with parameters
    hotels = requests.get(url, params=params)
    
#Convert data to json format
    hotels = hotels.json()

#Search for/select first hotel result from API results and store in 'Hotel Name' column
#If KeyError or IndexError present, print "Missing field/result... skipping." and continue iteration
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

#Display Hotels
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,Shilbania para
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,Hotel San Isidro Oficial
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,Hôtel & Restaurant DIANA Ambilobe
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,Le Monitel
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,Pousada Porto Praia
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36,Rasoolpur Barwa


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
#Set lat/longs from data frame as locations
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

#add layer to existing heatmap
fig.add_layer(markers)

#Show heatmap with markers
fig




Figure(layout=FigureLayout(height='420px'))